<img src = "../../Data/bgsedsc_0.jpg">

# Project: Decision Trees
### Ignacio Vidal-Quadras Costa


## Import pre-processed data

In [1]:
import pandas as pd
import os

seed = 54
# Training dataset
data=pd.read_csv('pre_processed_data.csv')
data = data.drop('Unnamed: 0', axis =1)
data.head()

,HOSPITAL_EXPIRE_FLAG,subject_id,hadm_id,icustay_id,HeartRate_Min,HeartRate_Max,HeartRate_Mean,SysBP_Min,SysBP_Max,SysBP_Mean,...,LOS,age,weekday,WEEKDAY,SEASON,HOLIDAY,hour,NIGHT,ETHNICS,num_diag
0,0,55440,195768,228357,89.0,145.0,121.043478,74.0,127.0,106.586957,...,4.5761,69.608219,6,Sunday,Summer,Non Holiday,10,0,OTHER,17
1,0,76908,126136,221004,63.0,110.0,79.117647,89.0,121.0,106.733333,...,0.7582,42.101370,0,Monday,Winter,Non Holiday,22,1,OTHER,3
2,0,95798,136645,296315,81.0,98.0,91.689655,88.0,138.0,112.785714,...,3.7626,68.210959,2,Wednesday,Autumn,Non Holiday,23,1,OTHER,14
3,0,40708,102505,245557,76.0,128.0,98.857143,84.0,135.0,106.972973,...,3.8734,74.961644,0,Monday,Autumn,Non Holiday,8,0,OTHER,39
4,0,28424,127337,225281,NaN,NaN,NaN,NaN,NaN,NaN,...,5.8654,79.424658,4,Friday,Autumn,Non Holiday,2,1,OTHER,69


## Import pre-processed test data

In [2]:
# Test dataset (to produce predictions)
data_test=pd.read_csv('pre_processed_data_test.csv')
data_test = data_test.drop('Unnamed: 0', axis =1)
data_test.sort_values('icustay_id').head()

,subject_id,hadm_id,icustay_id,HeartRate_Min,HeartRate_Max,HeartRate_Mean,SysBP_Min,SysBP_Max,SysBP_Mean,DiasBP_Min,...,FIRST_CAREUNIT,age,weekday,WEEKDAY,SEASON,HOLIDAY,hour,NIGHT,ETHNICS,num_diag
4930,93535,121562,200011,56.0,82.0,71.205128,123.0,185.0,156.411765,37.0,...,MICU,84.287671,3,Thursday,Winter,Non Holiday,20,1,OTHER,39
1052,30375,177945,200044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SICU,81.964384,6,Sunday,Winter,Non Holiday,16,0,OTHER,5
3412,73241,149216,200049,54.0,76.0,64.833333,95.0,167.0,114.545455,33.0,...,MICU,64.441096,5,Saturday,Winter,Non Holiday,22,1,OTHER,25
1725,99052,129142,200063,85.0,102.0,92.560976,91.0,131.0,108.365854,42.0,...,CSRU,37.095890,1,Tuesday,Summer,Non Holiday,23,1,OTHER,18
981,51698,190004,200081,82.0,133.0,94.323529,86.0,143.0,111.093750,47.0,...,CCU,69.920548,3,Thursday,Summer,Non Holiday,6,1,OTHER,24


## Import the necessary packages

In [3]:
%matplotlib inline
import matplotlib.pylab as plt

import pandas as pd
import numpy as np
import sklearn
import random
import time
import scipy
import datetime
import imblearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection  import ParameterGrid
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression
from mlens.ensemble import SuperLearner, BlendEnsemble, Subsemble, SequentialEnsemble

from sklearn.metrics import accuracy_score, mean_squared_error



np.random.seed(3123) # impose random seed for reproducibility

[MLENS] backend: threading


## Classifiaction of our variables and categorical transformation

### Classifiaction of variables
I will eliminate DOD, DISCHTIME because when the patient enters the hospital you cannot know this information (he is still alive)

Subject_id, hadm_id, icustay_id are eliminated because they have no explanatory power

DIAGNOSIS will be droped as I will use the ICD9_diagnosis (because it has fewer observations)

LOS is not included because you do not know it when you get into the hospital

In [4]:
y = data['LOS']

numericals = ['HeartRate_Min','HeartRate_Max','HeartRate_Mean','SysBP_Min',
       'SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean',
       'MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean', 'RespRate_Min',
       'RespRate_Max', 'RespRate_Mean', 'TempC_Min', 'TempC_Max', 'TempC_Mean',
       'SpO2_Min', 'SpO2_Max', 'SpO2_Mean', 'Glucose_Min', 'Glucose_Max',
       'Glucose_Mean', 'age', 'com_prob', 'num_diag']


categoricals = ['ADMISSION_TYPE', 'FIRST_CAREUNIT', 'GENDER', 'ETHNICS',
                'RELIGION', 'INSURANCE','NIGHT', 'WEEKDAY', 'SEASON', 'DIAG', 'HOLIDAY']
#Notice DIAG and com_prob do not exist yet, they will be added in the following steps

variables = numericals + categoricals

### Categorical encoding - Target encoding
I will encode the categoricals using target encoding. I have tried creating dummies but, besides being much slower, the algorythm works worse.

The target encoding is not done weith the target variable (in this case LOS) but with the probability of death, so it is not a proper target encoding but they are very similar as the more likely the client is to die, the more severe the illness is and the longer he will stay in the hospital.

Notice that on this code we don't use the same procedure for ICD9_diagnosis. This is because it might be the case that the diagnosis in the test sample is a new one (does not exist in the train set), in this case I will assign to this variable the general progability of dying.

In [5]:
for n in [i for i in categoricals if i != 'DIAG']:
    dict = data.groupby(n)['HOSPITAL_EXPIRE_FLAG'].mean().to_dict()  
    data[n] = data[n].apply(lambda x: dict[x])
    data_test[n] = data_test[n].apply(lambda x: dict[x])

In [6]:
diag_mean_dict = data.groupby('ICD9_diagnosis')['HOSPITAL_EXPIRE_FLAG'].mean().to_dict()

def get_diag(x):
    if x in diag_mean_dict:
        return diag_mean_dict[x]
    else: 
        return data.HOSPITAL_EXPIRE_FLAG.mean()

data['DIAG'] = data['ICD9_diagnosis'].apply(get_diag)
data_test['DIAG'] = data_test['ICD9_diagnosis'].apply(get_diag)


### Commulative probabilty per patient based on diagnosis
One patient can have come before to the hospital so he was already assigned a probability of death. In this variable we exactly capture that, if the patient has come a frist time and according to his diagnosis he was assigned a death probability of 0.01, then if he goes again and according to the diagnosis he has a probability of dying of 0.1, then the com_prob variable will account for the 0.11 commulative probability.

In [7]:
data['com_prob'] = data.sort_values(['subject_id', 'ADMITTIME']).groupby('subject_id')['DIAG'].cumsum()

data_test['com_prob']=data_test.sort_values(['subject_id', 'ADMITTIME']).groupby('subject_id')['DIAG'].cumsum()

## Select from the train and test only the variables we want

In [8]:
X_train = data[variables]
X_test = data_test[variables]

### Now let's make sure that both datasets have the same columns so that we can apply KNN

In [9]:
X_train.columns.isin(X_test.columns)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [10]:
X_train.columns

Index(['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean', 'SysBP_Min',
       'SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean',
       'MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean', 'RespRate_Min',
       'RespRate_Max', 'RespRate_Mean', 'TempC_Min', 'TempC_Max', 'TempC_Mean',
       'SpO2_Min', 'SpO2_Max', 'SpO2_Mean', 'Glucose_Min', 'Glucose_Max',
       'Glucose_Mean', 'age', 'com_prob', 'num_diag', 'ADMISSION_TYPE',
       'FIRST_CAREUNIT', 'GENDER', 'ETHNICS', 'RELIGION', 'INSURANCE', 'NIGHT',
       'WEEKDAY', 'SEASON', 'DIAG', 'HOLIDAY'],
      dtype='object')

In [11]:
col_to_use = X_train.columns[X_train.columns.isin(X_test.columns)]
len(col_to_use)

38

In [12]:
col_to_use = X_train.columns[X_train.columns.isin(X_test.columns)]
len(col_to_use)
X_train = X_train[col_to_use]
X_test = X_test[col_to_use]
print(X_test.shape)
print(X_train.shape)

(5221, 38)
(20885, 38)


## Dealing with the missing values

Previous in this assigment I tried different ways of handeling missing data.

I tried deleating all the rows with at least one missing value in the train data set and substituting the missing values in the test with the median of the varibale but my accuracy went down a lot.

Then I also tried the with the KNN method for missing data.

I also tried to subsitute them by just the mean.

Finally, the median turned out to be the most efficient one.


In [13]:
X_train.fillna(X_train.median(), inplace=True)
X_test.fillna(X_test.median(), inplace=True)

## Feature engenieering
##### (Find the code in the support_file)

I will not explore the skewness and kurtosis of the numerical varibales because the models I am implementing are ¡ and XGBoost and random forest, since decision trees do not require normalization of their inputs and both random forests and XGBoost are a variation of decision trees, they do not require normalization of their imnputs either. As consequence I will not apply logs or any other normalization technique.

I have droped categorical data so I will note use feature slips.

I tried to delete records with ourliers from the data set but the outcome was not good. Defined as below 0.1 quantile and above 0.9 quantile.

For varibale reduction I explored at the correlation matrix and tried to delete some correlated varibales but did not end up beeing helpul.

## Generating the predictions with the grid search in XGBoost

## XGBoost

In [14]:
%%time

model = XGBRegressor()

grid_values = {
    'n_estimators' : [100,200],
    'learning_rate': [0.1], 
    'max_depth': [3],
    'random_state' : [seed],
    'sampling_method': ['uniform', 'gradient_based']
}


grid_xgb_acc = GridSearchCV(model, param_grid = grid_values, cv=5, scoring='neg_mean_squared_error')

grid_xgb_acc.fit(X_train, y)

y_pred_acc = grid_xgb_acc.predict(X_test)

print("MSE::{}".format(grid_xgb_acc.best_score_))
print("Best Hyperparameters::\n{}".format(grid_xgb_acc.best_params_))

MSE::-22.74404293221945
Best Hyperparameters::
{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'random_state': 54, 'sampling_method': 'uniform'}
CPU times: user 1min 15s, sys: 122 ms, total: 1min 15s
Wall time: 12.7 s


In [15]:
importances = grid_xgb_acc.best_estimator_.feature_importances_
important_features = pd.Series(data=importances/importances.max() ,index=X_train.columns).sort_values(ascending=False)
important_features

num_diag          1.000000
TempC_Max         0.762976
DIAG              0.642572
Glucose_Max       0.618550
HeartRate_Max     0.530426
RespRate_Max      0.488970
SysBP_Min         0.441329
SpO2_Max          0.436577
FIRST_CAREUNIT    0.407280
DiasBP_Min        0.374537
TempC_Mean        0.320553
SpO2_Min          0.304181
SysBP_Max         0.280317
RespRate_Mean     0.275251
Glucose_Min       0.271678
SpO2_Mean         0.266647
age               0.262331
MeanBP_Max        0.262001
TempC_Min         0.249088
SysBP_Mean        0.236052
com_prob          0.229403
MeanBP_Min        0.221853
DiasBP_Mean       0.221654
WEEKDAY           0.214883
MeanBP_Mean       0.212866
Glucose_Mean      0.211198
DiasBP_Max        0.196942
HeartRate_Min     0.193808
HeartRate_Mean    0.183663
INSURANCE         0.176269
NIGHT             0.167249
RespRate_Min      0.164482
ADMISSION_TYPE    0.138974
RELIGION          0.089468
GENDER            0.087909
ETHNICS           0.000000
SEASON            0.000000
H

After exploring the importances I will drop the ones with importance below 0.05, which are the ones that have no importance on the model. Season, Holiday and Ethincs.
After this I will run again XGBoost with the selected model

In [18]:
X_train = X_train.drop(['SEASON', 'HOLIDAY', 'ETHNICS'], axis = 1)
X_test = X_test.drop(['SEASON', 'HOLIDAY', 'ETHNICS'], axis = 1)

In [19]:
%%time

model = XGBRegressor()

grid_values = {
    'n_estimators' : [100,200],
    'learning_rate': [0.1], 
    'max_depth': [3],
    'random_state' : [seed],
    'sampling_method': ['uniform', 'gradient_based']
}


grid_xgb_acc = GridSearchCV(model, param_grid = grid_values, cv=5, scoring='neg_mean_squared_error')

grid_xgb_acc.fit(X_train, y)

y_pred_acc = grid_xgb_acc.predict(X_test)

print("MSE::{}".format(grid_xgb_acc.best_score_))
print("Best Hyperparameters::\n{}".format(grid_xgb_acc.best_params_))

MSE::-22.709509039589673
Best Hyperparameters::
{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'random_state': 54, 'sampling_method': 'uniform'}
CPU times: user 1min 19s, sys: 121 ms, total: 1min 19s
Wall time: 13.4 s


To sum up, after having tried differnt approaches to the model applying the models and techniques we have learned in class, the XGBoost after having selected these variables with an importance abpve 0.05 ended up to be the best one for predicitons. This XGBoost is using 200 number of gradient boos trees, a boosting learning rate of 0.1 and a maximium tree depth of 3, and where each training instance has the same probability of being selected.

In [20]:
pred_df = pd.DataFrame(data_test['icustay_id'].copy())
pred_df['LOS'] = y_pred_acc
pred_df.to_csv('XGB_Prediction_kaggle', index=False)